## Valorant Decision Tree Model

In [25]:
# Initial imports
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import accuracy_score 
from sqlalchemy import create_engine, Column, Integer, String, Boolean, Float, ForeignKey

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from kerastuner.tuners import RandomSearch
# from tensorflow.keras.callbacks import EarlyStopping


## Loading and Preparing Agent Data

In [26]:
import sqlite3

# Create database connection
engine = create_engine('sqlite:///SQLite/Valorant.sqlite')

In [27]:
# Querying and creating Dataframe
query_1 = """
    SELECT agents.agent, round_results.roundResult, player_stats.competitiveTier, player_stats.roundsPlayed, player_stats.score, match_info.gameLengthMillis, player_stats.assists, player_stats.kills, player_stats.deaths, team_stats.numPoints
    FROM agents
    INNER JOIN round_results ON agents.id = round_results.id
    INNER JOIN player_stats ON agents.id = player_stats.id
    INNER JOIN player_kills ON agents.id = player_kills.id 
    INNER JOIN match_info ON agents.id = match_info.id
    INNER JOIN team_stats ON agents.id = team_stats.id
"""

agent_df = pd.read_sql(query_1, engine)
agent_df

,agent,roundResult,competitiveTier,roundsPlayed,score,gameLengthMillis,assists,kills,deaths,numPoints
0,Gekko,Bomb detonated,IRON_1,22,1140,2046675,0,4,8,9
1,Fade,Eliminated,IRON_2,22,6855,2083360,3,22,16,13
2,Breach,Eliminated,UNRANKED,22,3669,2155617,2,12,17,13
3,Deadlock,Eliminated,IRON_2,22,4817,2307563,5,17,13,8
4,Raze,Eliminated,IRON_2,22,2580,2453157,4,9,11,8
5,Chamber,Bomb detonated,IRON_3,22,2875,1866841,10,10,10,13
6,KAY/O,Eliminated,IRON_3,22,1723,2228659,3,4,18,13
7,Skye,Bomb defused,IRON_3,22,7816,1732559,4,27,12,9
8,Cypher,Bomb defused,IRON_1,22,3586,1916596,3,12,10,13
9,Sova,Eliminated,IRON_3,22,3974,1956727,3,14,16,10


In [28]:
from sklearn.preprocessing import LabelEncoder

# Initializing LabelEncoder for roundResult and competitiveTier
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()

# Applying label encoding
agent_df["roundResult_encoded"] = label_encoder1.fit_transform(agent_df["roundResult"])
agent_df["competitiveTier_encoded"] = label_encoder2.fit_transform(agent_df["competitiveTier"])

# Dropping and rearranging columns
columns_to_drop = ["roundResult", "competitiveTier"]
agent_df_refined = agent_df.drop(columns=columns_to_drop)

agent_df_refined

,agent,roundsPlayed,score,gameLengthMillis,assists,kills,deaths,numPoints,roundResult_encoded,competitiveTier_encoded
0,Gekko,22,1140,2046675,0,4,8,9,1,5
1,Fade,22,6855,2083360,3,22,16,13,2,6
2,Breach,22,3669,2155617,2,12,17,13,2,10
3,Deadlock,22,4817,2307563,5,17,13,8,2,6
4,Raze,22,2580,2453157,4,9,11,8,2,6
5,Chamber,22,2875,1866841,10,10,10,13,1,7
6,KAY/O,22,1723,2228659,3,4,18,13,2,7
7,Skye,22,7816,1732559,4,27,12,9,0,7
8,Cypher,22,3586,1916596,3,12,10,13,0,5
9,Sova,22,3974,1956727,3,14,16,10,2,7


In [29]:
# Isolating our independent variable X
X = agent_df_refined.drop("agent", axis=1).values
X[:5]

array([[     22,    1140, 2046675,       0,       4,       8,       9,
              1,       5],
       [     22,    6855, 2083360,       3,      22,      16,      13,
              2,       6],
       [     22,    3669, 2155617,       2,      12,      17,      13,
              2,      10],
       [     22,    4817, 2307563,       5,      17,      13,       8,
              2,       6],
       [     22,    2580, 2453157,       4,       9,      11,       8,
              2,       6]])

In [30]:
# Isolating our dependent variable y
y = agent_df_refined["agent"].values
y

array(['Gekko', 'Fade', 'Breach', 'Deadlock', 'Raze', 'Chamber', 'KAY/O',
       'Skye', 'Cypher', 'Sova', 'Sova', 'Killjoy', 'Harbor', 'Viper',
       'Phoenix', 'Astra', 'Brimstone', 'Neon', 'Yoru', 'Sage', 'Reyna',
       'Omen', 'Jett', 'Gekko', 'Fade', 'Breach', 'Deadlock', 'Raze',
       'Chamber', 'KAY/O', 'Skye', 'Cypher', 'Sova', 'Sova', 'Killjoy',
       'Harbor', 'Viper', 'Phoenix', 'Astra', 'Brimstone', 'Neon', 'Yoru',
       'Sage', 'Reyna', 'Omen', 'Jett'], dtype=object)

## Creating the Decision Tree Model

In [31]:
# Spliting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=78)

In [32]:
# Create a StandardScaler instance
scaler = StandardScaler()

In [33]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [34]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Fitting the Decision Tree Model

In [35]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [36]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

## Making Predictions and Model Evaluation

In [37]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [38]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

ValueError: Shape of passed values is (14, 14), indices imply (2, 2)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))